In [1]:
%pylab inline --no-import-all
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'
import sys
import os

folder = '../../../../crispy'
print(folder)
if folder not in sys.path: sys.path.append(folder)

from crispy.params import Params
folder = '../../../crispy'
par = Params(folder)

from crispy.tools.initLogger import getLogger
log = getLogger('crispy')
from crispy.tools.imgtools import scale2imgs,bowtie
from crispy.tools.image import Image
from astropy.io import fits
import astropy.units as u
from crispy.tools.postprocessing import mf
lamc=660.
BW = 0.18
par.nonoise=True
sampling=2.0
par.lenslet_sampling=1./sampling
folder = '/Users/mrizzo/IFS/OS5_SIM_'+str(sampling)


Populating the interactive namespace from numpy and matplotlib
../../../../crispy


### Implementation of a matched filter that preserves contrast